# Simplified Multirate Kalman Filter Design via LMI Optimization

**Author:** Hiroshi Okajima  
**Date:** February 2025

## Description
Simplified version with state dimension n=1, output m=1, period N=6.  
This allows easy modification to periods N=1, 2, 3, 6 by adjusting $S_k$.

## System
$$x(k+1) = Ax(k) + Bu(k) + w(k), \quad w \sim \mathcal{N}(0, Q)$$
$$y(k) = S_k C x(k) + S_k v(k), \quad v \sim \mathcal{N}(0, R)$$

## Reference
H. Okajima, "LMI Optimization Based Multirate Steady-State Kalman Filter Design," IEEE Access, 2025.

## 0. Install Required Packages

In [ ]:
!pip install numpy scipy matplotlib cvxpy

## 1. Import Libraries

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are

# For better plot display in notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"CVXPY version: {cp.__version__}")

## 2. System Definition

In [ ]:
# System parameters
n = 1      # State dimension
p = 1      # Input dimension  
m = 1      # Output dimension
N = 6      # Period (can test N=1,2,3,6 by changing S pattern)
dt = 0.1   # Sampling time [s]

# First-order system: position tracking
A = np.array([[0.95]])   # Stable system (eigenvalue < 1)
B = np.array([[0.1]])    # Input gain
C = np.array([[1.0]])    # Full state observation when available

# Noise covariances
Q = np.array([[0.1]])    # Process noise variance
R = np.array([[1.0]])    # Measurement noise variance

print("="*60)
print("System Parameters")
print("="*60)
print(f"State dimension: n = {n}")
print(f"Output dimension: m = {m}")
print(f"Period: N = {N}")
print(f"System matrix A = {A[0,0]:.3f} (eigenvalue)")
print(f"Process noise Q = {Q[0,0]:.3f}")
print(f"Measurement noise R = {R[0,0]:.3f}")

## 3. Measurement Pattern Selection

Choose one of the following patterns by changing `pattern_type`:

| Pattern | Description | Effective Period |
|---------|-------------|------------------|
| 1 | Sensor every 6 steps | N=6 |
| 2 | Sensor every 3 steps | N=3 |
| 3 | Sensor every 2 steps | N=2 |
| 4 | Sensor every step | N=1 (Standard KF) |

In [ ]:
def create_measurement_pattern(N, pattern_type=1):
    """
    Create measurement pattern S_k
    """
    if pattern_type == 1:
        S = [np.array([[1]]), np.array([[0]]), np.array([[0]]),
             np.array([[0]]), np.array([[0]]), np.array([[0]])]
    elif pattern_type == 2:
        S = [np.array([[1]]), np.array([[0]]), np.array([[0]]),
             np.array([[1]]), np.array([[0]]), np.array([[0]])]
    elif pattern_type == 3:
        S = [np.array([[1]]), np.array([[0]]), np.array([[1]]),
             np.array([[0]]), np.array([[1]]), np.array([[0]])]
    elif pattern_type == 4:
        S = [np.array([[1]]) for _ in range(N)]
    return S

# ====== SELECT PATTERN HERE ======
pattern_type = 1  # Change to 1, 2, 3, or 4
# =================================

S = create_measurement_pattern(N, pattern_type)

print(f"\nSelected Pattern Type: {pattern_type}")
print("\nMeasurement Pattern S_k:")
for i in range(N):
    status = "(measurement available)" if S[i][0,0] == 1 else "(no measurement)"
    print(f"  S_{i} = {int(S[i][0,0])} {status}")

## 4. Cyclic Reformulation Construction

### Cyclic System Matrix $\check{A}$ (Equation 24)
$$\check{A} = \begin{bmatrix} 0 & 0 & \cdots & 0 & A \\ A & 0 & \cdots & 0 & 0 \\ 0 & A & \cdots & 0 & 0 \\ \vdots & \vdots & \ddots & \vdots & \vdots \\ 0 & 0 & \cdots & A & 0 \end{bmatrix}$$

In [ ]:
def construct_cyclic_system(A, B, C, Q, R, S, N):
    """Construct cyclic reformulation of the multirate system"""
    n = A.shape[0]
    p = B.shape[1]
    m = C.shape[0]
    
    # A_cyc: Nn × Nn cyclic matrix
    A_cyc = np.zeros((N*n, N*n))
    A_cyc[0:n, (N-1)*n:N*n] = A
    for i in range(1, N):
        A_cyc[i*n:(i+1)*n, (i-1)*n:i*n] = A
    
    # B_cyc
    B_cyc = np.zeros((N*n, N*p))
    B_cyc[0:n, (N-1)*p:N*p] = B
    for i in range(1, N):
        B_cyc[i*n:(i+1)*n, (i-1)*p:i*p] = B
    
    # C_cyc: block-diagonal
    C_cyc = np.zeros((N*m, N*n))
    for i in range(N):
        C_cyc[i*m:(i+1)*m, i*n:(i+1)*n] = S[i] @ C
    
    # Q_cyc, R_cyc
    Q_cyc = np.kron(np.eye(N), Q)
    R_cyc = np.zeros((N*m, N*m))
    for i in range(N):
        R_cyc[i*m:(i+1)*m, i*m:(i+1)*m] = S[i] @ R @ S[i].T
    
    return A_cyc, B_cyc, C_cyc, Q_cyc, R_cyc

A_cyc, B_cyc, C_cyc, Q_cyc, R_cyc = construct_cyclic_system(A, B, C, Q, R, S, N)

print("="*60)
print("Cyclic Reformulation")
print("="*60)
print(f"\nA_cyc: {A_cyc.shape}")
print(A_cyc)
print(f"\nC_cyc: {C_cyc.shape}")
print(C_cyc)
print(f"\nR_cyc diagonal: {np.diag(R_cyc)}")

## 5. Key Observation: $\check{R}$ is Semidefinite (Remark 3.1)

In [ ]:
rank_R = np.linalg.matrix_rank(R_cyc)
print(f"R_cyc rank: {rank_R} / {N*m}")
if rank_R < N*m:
    print("\n*** R_cyc is SEMIDEFINITE (not positive definite) ***")
    print("*** Standard DARE cannot be used! ***")
    print("*** LMI-based approach is necessary ***")

## 6. Observability Check

In [ ]:
def check_observability(A_cyc, C_cyc):
    n_cyc = A_cyc.shape[0]
    O = C_cyc.copy()
    Ak = np.eye(n_cyc)
    for i in range(1, n_cyc):
        Ak = Ak @ A_cyc
        O = np.vstack([O, C_cyc @ Ak])
    return np.linalg.matrix_rank(O), np.linalg.matrix_rank(O) == n_cyc

rank_O, is_obs = check_observability(A_cyc, C_cyc)
print(f"Observability matrix rank: {rank_O} / {N*n}")
print(f"Result: {'Observable ✓' if is_obs else 'NOT Observable ✗'}")

## 7. LMI-based Filter Design (Dual LQR Formulation)

### LMI Formulation (Section IV, Equation 39)

$$\begin{bmatrix} \check{X} & (\check{A}^T\check{X} + \check{C}^T\check{Y})^T & \check{Q}^{1/2}\check{X} & \check{R}^{1/2}\check{Y} \\ \check{A}^T\check{X} + \check{C}^T\check{Y} & \check{X} & 0 & 0 \\ \check{X}\check{Q}^{1/2} & 0 & I & 0 \\ \check{Y}^T\check{R}^{1/2} & 0 & 0 & I \end{bmatrix} \succeq 0$$

### Kalman Gain Extraction
$$\check{K} = (-\check{P}\check{Y})^T \quad \text{where} \quad \check{P} = \check{X}^{-1}$$

In [ ]:
def construct_sqrt_matrices(Q, R, S, N, epsilon=1e-8):
    """Construct Q_cyc_sqrt and R_cyc_sqrt"""
    n = Q.shape[0]
    m = R.shape[0]
    Q_sqrt = np.linalg.cholesky(Q)
    
    # Q_cyc_sqrt: cyclic structure
    Q_cyc_sqrt = np.zeros((N*n, N*n))
    Q_cyc_sqrt[0:n, (N-1)*n:N*n] = Q_sqrt
    for i in range(1, N):
        Q_cyc_sqrt[i*n:(i+1)*n, (i-1)*n:i*n] = Q_sqrt
    
    # R_cyc_sqrt: block diagonal with regularization
    R_cyc_sqrt = np.zeros((N*m, N*m))
    for i in range(N):
        R_block = S[i] @ R @ S[i].T + epsilon * np.eye(m)
        R_cyc_sqrt[i*m:(i+1)*m, i*m:(i+1)*m] = np.linalg.cholesky(R_block)
    
    return Q_cyc_sqrt, R_cyc_sqrt

epsilon = 1e-8
Q_cyc_sqrt, R_cyc_sqrt = construct_sqrt_matrices(Q, R, S, N, epsilon)
print(f"R_cyc regularization: epsilon = {epsilon:.1e}")

In [ ]:
def solve_lmi_kalman_filter(A_cyc, C_cyc, Q_cyc_sqrt, R_cyc_sqrt, epsilon=1e-6):
    """Solve LMI optimization for multirate Kalman filter (Equation 39)"""
    n_cyc = A_cyc.shape[0]
    m_cyc = C_cyc.shape[0]
    
    Ad = A_cyc.T
    Bd = C_cyc.T
    
    X = cp.Variable((n_cyc, n_cyc), symmetric=True)
    Y = cp.Variable((m_cyc, n_cyc))
    W = cp.Variable((n_cyc, n_cyc), symmetric=True)
    
    constraints = []
    
    # LMI #1 (Equation 39): using transposed sqrt matrices
    block_21 = Ad @ X + Bd @ Y
    block_31 = Q_cyc_sqrt.T @ X    # (Q^{1/2})^T * X
    block_41 = R_cyc_sqrt.T @ Y    # (R^{1/2})^T * Y
    
    LMI1 = cp.bmat([
        [X, block_21.T, block_31.T, block_41.T],
        [block_21, X, np.zeros((n_cyc, n_cyc)), np.zeros((n_cyc, m_cyc))],
        [block_31, np.zeros((n_cyc, n_cyc)), np.eye(n_cyc), np.zeros((n_cyc, m_cyc))],
        [block_41, np.zeros((m_cyc, n_cyc)), np.zeros((m_cyc, n_cyc)), np.eye(m_cyc)]
    ])
    constraints.append(LMI1 >> 0)
    constraints.append(X >> epsilon * np.eye(n_cyc))
    
    # LMI #3: W >= X^{-1}
    LMI3 = cp.bmat([[W, np.eye(n_cyc)], [np.eye(n_cyc), X]])
    constraints.append(LMI3 >> 0)
    
    problem = cp.Problem(cp.Minimize(cp.trace(W)), constraints)
    problem.solve(solver=cp.SCS, verbose=False, max_iters=10000, eps=1e-9)
    
    if problem.status not in ['optimal', 'optimal_inaccurate']:
        raise ValueError(f"LMI failed: {problem.status}")
    
    X_opt, Y_opt, W_opt = X.value, Y.value, W.value
    P_ss = np.linalg.inv(X_opt)
    
    # Kalman gain: K_ss = (-P_ss @ Y_opt).T (matches MATLAB)
    K_ss = (-P_ss @ Y_opt).T
    
    return X_opt, Y_opt, W_opt, K_ss, P_ss, problem.value

print("Solving LMI optimization...")
X_opt, Y_opt, W_opt, K_ss, P_ss, opt_trace = solve_lmi_kalman_filter(
    A_cyc, C_cyc, Q_cyc_sqrt, R_cyc_sqrt
)

print(f"\nLMI Results:")
print(f"  trace(W) = {opt_trace:.6f}")
print(f"  trace(P) = {np.trace(P_ss):.6f}")

## 8. Stability Analysis

In [ ]:
A_cl = A_cyc - K_ss @ C_cyc
eig_cl = np.linalg.eigvals(A_cl)
max_eig = np.max(np.abs(eig_cl))

print(f"Max |eigenvalue| of A_cyc - K_ss*C_cyc: {max_eig:.6f}")
print(f"Result: {'STABLE ✓' if max_eig < 1 else 'UNSTABLE ✗'}")

## 9. Extract Periodic Kalman Gains (Equation 44)

In [ ]:
def extract_periodic_gains(K_ss, N, n, m):
    """Extract periodic gains following Equation 44"""
    L = []
    L.append(K_ss[1*n:2*n, 0*m:1*m])  # L_0
    for k in range(1, N-1):
        L.append(K_ss[(k+1)*n:(k+2)*n, k*m:(k+1)*m])
    L.append(K_ss[0*n:1*n, (N-1)*m:N*m])  # L_{N-1}
    return L

L = extract_periodic_gains(K_ss, N, n, m)

print("Periodic Kalman Gains L_k:")
for k in range(N):
    status = "(measurement)" if S[k][0,0] == 1 else "(no measurement)"
    print(f"  L_{k} = {L[k][0,0]:.6f}  {status}")

## 10. Simulation

In [ ]:
def simulate(A, B, C, Q, R, S, L, T, x0=5.0, seed=42):
    np.random.seed(seed)
    N = len(S)
    n, m = A.shape[0], C.shape[0]
    
    x_true = np.zeros((n, T))
    x_true[:, 0] = x0
    y_obs = np.zeros((m, T))
    u = 0.5 * np.sin(0.1 * np.arange(T))
    
    Q_sqrt, R_sqrt = np.linalg.cholesky(Q), np.linalg.cholesky(R)
    
    for k in range(T - 1):
        x_true[:, k+1] = A @ x_true[:, k] + B.flatten() * u[k] + Q_sqrt @ np.random.randn(n)
    for k in range(T):
        y_obs[:, k] = C @ x_true[:, k] + R_sqrt @ np.random.randn(m)
    
    x_hat = np.zeros((n, T))
    x_hat[:, 0] = x_true[:, 0]
    
    for k in range(T - 1):
        x_pred = A @ x_hat[:, k] + B.flatten() * u[k]
        idx = k % N
        if S[idx][0, 0] == 1:
            x_hat[:, k+1] = x_pred + L[idx] @ (y_obs[:, k+1] - C @ x_pred)
        else:
            x_hat[:, k+1] = x_pred
    
    return x_true, x_hat, y_obs, u

T = 100
x_true, x_hat, y_obs, u = simulate(A, B, C, Q, R, S, L, T)

error = x_true - x_hat
rmse = np.sqrt(np.mean(error**2))
print(f"RMSE: {rmse:.4f}")

## 11. Comparison with Standard Kalman Filter

In [ ]:
P_std = solve_discrete_are(A.T, C.T, Q, R)
K_std = A @ P_std @ C.T @ np.linalg.inv(C @ P_std @ C.T + R)

x_hat_std = np.zeros((n, T))
x_hat_std[:, 0] = x_true[:, 0]
for k in range(T - 1):
    x_pred = A @ x_hat_std[:, k] + B.flatten() * u[k]
    x_hat_std[:, k+1] = x_pred + K_std @ (y_obs[:, k+1] - C @ x_pred)

rmse_std = np.sqrt(np.mean((x_true - x_hat_std)**2))
n_obs = sum([1 for k in range(T) if S[k % N][0,0] == 1])

print(f"Standard KF: K = {K_std[0,0]:.6f}, P = {P_std[0,0]:.6f}")
print(f"\nMultirate KF RMSE: {rmse:.4f}")
print(f"Standard KF RMSE:  {rmse_std:.4f}")
print(f"Observations used: {n_obs}/{T} ({100*n_obs/T:.1f}%)")

## 12. Visualization

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Plot 1: State estimation
ax1 = axes[0, 0]
ax1.plot(range(T), x_true.flatten(), 'k-', lw=2, label='True')
ax1.plot(range(T), x_hat.flatten(), 'b--', lw=1.5, label='Estimated')
obs_t = [k for k in range(T) if S[k % N][0, 0] == 1]
ax1.plot(obs_t, y_obs.flatten()[obs_t], 'ro', ms=4, label='Obs')
ax1.set_xlabel('k'); ax1.set_ylabel('x'); ax1.set_title('State Estimation')
ax1.legend(); ax1.grid(True)

# Plot 2: Error
ax2 = axes[0, 1]
ax2.plot(range(T), error.flatten(), lw=1.5)
ax2.axhline(0, color='k', ls='--')
ax2.set_xlabel('k'); ax2.set_ylabel('Error')
ax2.set_title(f'Estimation Error (RMSE={rmse:.4f})'); ax2.grid(True)

# Plot 3: Pattern
ax3 = axes[0, 2]
pattern = [S[k % N][0, 0] for k in range(T)]
ax3.stem(range(T), pattern)
ax3.set_xlabel('k'); ax3.set_ylabel('$S_k$')
ax3.set_title('Measurement Pattern'); ax3.set_ylim([-0.1, 1.1]); ax3.grid(True)

# Plot 4: Eigenvalues
ax4 = axes[1, 0]
theta = np.linspace(0, 2*np.pi, 100)
ax4.plot(np.cos(theta), np.sin(theta), 'k--', lw=1)
ax4.plot(np.real(eig_cl), np.imag(eig_cl), 'bo', ms=10)
ax4.set_aspect('equal'); ax4.set_xlabel('Real'); ax4.set_ylabel('Imag')
ax4.set_title(f'Eigenvalues (max|λ|={max_eig:.4f})'); ax4.grid(True)

# Plot 5: Gains
ax5 = axes[1, 1]
L_vals = [L[k][0, 0] for k in range(N)]
colors = ['g' if S[k][0,0]==1 else 'gray' for k in range(N)]
ax5.bar(range(N), L_vals, color=colors)
ax5.set_xlabel('k mod N'); ax5.set_ylabel('$L_k$')
ax5.set_title('Periodic Gains'); ax5.grid(True)

# Plot 6: K_ss
ax6 = axes[1, 2]
im = ax6.imshow(K_ss, aspect='auto', cmap='viridis')
plt.colorbar(im, ax=ax6)
ax6.set_xlabel('Col'); ax6.set_ylabel('Row'); ax6.set_title('$K_{ss}$ Matrix')

plt.suptitle(f'Multirate KF: n={n}, m={m}, N={N}, Pattern={pattern_type}', fontsize=14)
plt.tight_layout()
plt.show()

## 13. Compare All Patterns

In [ ]:
print("Pattern Comparison:")
print("-"*70)
print(f"{'Pattern':<10} {'Obs%':<10} {'L_0':<12} {'max|λ|':<12} {'RMSE':<10}")
print("-"*70)

for pt in [1, 2, 3, 4]:
    S_t = create_measurement_pattern(N, pt)
    A_c, B_c, C_c, Q_c, R_c = construct_cyclic_system(A, B, C, Q, R, S_t, N)
    Q_s, R_s = construct_sqrt_matrices(Q, R, S_t, N, epsilon)
    
    _, _, _, K_t, P_t, _ = solve_lmi_kalman_filter(A_c, C_c, Q_s, R_s)
    L_t = extract_periodic_gains(K_t, N, n, m)
    
    A_cl_t = A_c - K_t @ C_c
    max_e = np.max(np.abs(np.linalg.eigvals(A_cl_t)))
    
    x_tr, x_h, y_o, u_t = simulate(A, B, C, Q, R, S_t, L_t, T)
    rmse_t = np.sqrt(np.mean((x_tr - x_h)**2))
    obs_pct = 100 * sum([S_t[k % N][0,0] for k in range(T)]) / T
    
    print(f"{pt:<10} {obs_pct:<10.1f} {L_t[0][0,0]:<12.6f} {max_e:<12.6f} {rmse_t:<10.4f}")

## Summary

This notebook demonstrated LMI-based multirate Kalman filter design.

**Key Points:**
1. $\check{R}$ is semidefinite when sensors operate at different rates
2. LMI approach handles semidefinite $\check{R}$ naturally
3. Kalman gain: $\check{K} = (-\check{P}\check{Y})^T$ where $\check{P} = \check{X}^{-1}$